In [108]:
import os
import gradio as gr
import openai
import anthropic
from IPython.display import Markdown, display, update_display
from pprint import pprint as print
import json

In [109]:
system_message = "You are a shady ex-con from the 19th century. You help rich patrons secure their homes. You speak in no more than two sentences at a time, and use heavy 19th century British street slang."

In [110]:
openai.api_key = os.getenv("OPENAI_API_KEY")
GPT_MODEL = 'gpt-4o-mini'

In [111]:
def call_gpt(prompt, history):
    relevant_system_message = system_message
    if "police" or "cops" in prompt:
        relevant_system_message += "Any mention of police officers will have you respond only in derogatory remarks about police, but in pig latin"
    messages = [{"role":"system","content":relevant_system_message}]+ history +[{"role":"user","content":prompt}]
    
    stream = openai.chat.completions.create(messages=messages,model=GPT_MODEL,stream=True, )
    
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [112]:
ticket_prices = {"london": 799, "paris": 899, "tokyo": 1400, "berlin": 499}
destination_taxes = {"london": 3000, "paris": 3200, "tokyo": 6800, "berlin": 3400}

def get_ticket_price(destination_city,agency_tax):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, 0) + destination_taxes.get(city,0)+ agency_tax

In [113]:
total_price= get_ticket_price('paris', 200)
total_price

'Tool get_ticket_price called for paris'


4299

In [114]:
properties = {}
def create_properties(arg_name,arg_type,description) -> dict:   
    """takes arg names and packages them as properties for openai"""
    properties[f'{arg_name}'] = {"type":arg_type, "description":description}
    return properties

In [115]:

    
def create_function_package(function_name,function_description,properties) -> dict: 
    tool_function = {
        "name": function_name,
        "description": function_description,
        "parameters": {
            "type": "object",
            "properties": properties,
            
            "required": ["destination_city"],
            "additionalProperties": False
        },
    }
    return tool_function



In [116]:
function_name = "get_ticket_price"
function_description = "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'"

properties = create_properties('destination_city','string','The city that the customer wants to travel to')
properties = create_properties('agency_tax','integer','the fixed cost for each visit to a particular city')




tool_function = create_function_package(function_name,function_description,properties)
tools = [{"type": "function", "function": tool_function}]

In [117]:
j_tools = json.dumps(tool_function, indent=4)

with open('tools.json','w') as file:
    file.write(j_tools)
    print(file)

<_io.TextIOWrapper name='tools.json' mode='w' encoding='cp1252'>
